## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Waingapu,ID,-9.6567,120.2641,79.74,81,99,5.08,overcast clouds
1,1,Khatanga,RU,71.9667,102.5000,-7.24,92,100,14.23,overcast clouds
2,2,Pisco,PE,-13.7000,-76.2167,69.85,73,0,18.41,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,53.26,35,0,10.36,clear sky
4,4,Hobart,AU,-42.8794,147.3294,58.86,67,0,10.36,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  75
What is the maximum temperature you would like for your trip?  90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Waingapu,ID,-9.6567,120.2641,79.74,81,99,5.08,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,75.79,79,11,15.21,light rain
13,13,Jega,NG,12.2175,4.3792,79.41,12,90,9.64,overcast clouds
17,17,Cayenne,GF,4.9333,-52.3333,82.44,83,75,4.61,broken clouds
21,21,Torit,SS,4.4133,32.5678,76.19,64,87,2.10,overcast clouds
22,22,Uarini,BR,-2.9900,-65.1083,80.55,94,100,0.72,overcast clouds
23,23,Trinidad,BO,-14.8333,-64.9000,87.78,70,20,6.91,few clouds
27,27,Georgetown,MY,5.4112,100.3354,76.95,94,20,2.30,few clouds
29,29,Gladstone,AU,-23.8500,151.2500,80.53,74,75,11.50,broken clouds
30,30,Avarua,CK,-21.2078,-159.7750,82.45,83,20,11.50,light rain


In [21]:
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 0 to 719
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              188 non-null    int64  
 1   City                 188 non-null    object 
 2   Country              188 non-null    object 
 3   Lat                  188 non-null    float64
 4   Lng                  188 non-null    float64
 5   Max Temp             188 non-null    float64
 6   Humidity             188 non-null    int64  
 7   Cloudiness           188 non-null    int64  
 8   Wind Speed           188 non-null    float64
 9   Current Description  188 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 16.2+ KB


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                188
City                   188
Country                188
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, current description and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Waingapu,ID,79.74,overcast clouds,-9.6567,120.2641,
5,Rikitea,PF,75.79,light rain,-23.1203,-134.9692,
13,Jega,NG,79.41,overcast clouds,12.2175,4.3792,
17,Cayenne,GF,82.44,broken clouds,4.9333,-52.3333,
21,Torit,SS,76.19,overcast clouds,4.4133,32.5678,
22,Uarini,BR,80.55,overcast clouds,-2.9900,-65.1083,
23,Trinidad,BO,87.78,few clouds,-14.8333,-64.9000,
27,Georgetown,MY,76.95,few clouds,5.4112,100.3354,
29,Gladstone,AU,80.53,broken clouds,-23.8500,151.2500,
30,Avarua,CK,82.45,light rain,-21.2078,-159.7750,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Waingapu,ID,79.74,overcast clouds,-9.6567,120.2641,Pondok Wisata Elim
5,Rikitea,PF,75.79,light rain,-23.1203,-134.9692,People ThankYou
13,Jega,NG,79.41,overcast clouds,12.2175,4.3792,ALH. Nasiru Ibrahim
17,Cayenne,GF,82.44,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
21,Torit,SS,76.19,overcast clouds,4.4133,32.5678,Diplomat Guest House


In [26]:
clean_hotel_df = hotel_df

In [27]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))